# PPI Networks and Structural Interactomics

Graphein allows the retrieval of protein-protein interaction data from two widely-used sources: [STRING](https://string-db.org/) and [BioGrid](https://thebiogrid.org/). In conjunction with the structural graph utilities in Graphein, this can enable the development of geometric deep learning methods for structural interactomics.


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/a-r-j/graphein/blob/master/notebooks/ppi_graph.ipynb)

In [ ]:
# Install Graphein if necessary
# !pip install graphein

In [ ]:
import logging
logging.getLogger('urllib3').setLevel(level=logging.CRITICAL)
#logging.getLogger('matplotlib').setLevel(logging.CRITICAL)


## Config
Global parameters are stored in Config objects. We have a global [`PPIGraphConfig`](https://graphein.ai/modules/graphein.ppi.html#graphein.ppi.config.PPIGraphConfig) that contains a [`STRINGConfig`](https://graphein.ai/modules/graphein.ppi.html#graphein.ppi.config.STRINGConfig) and a [`BioGridConfig`](https://graphein.ai/modules/graphein.ppi.html#graphein.ppi.config.BioGridConfig) for parameters relating to each of the two sources.


In [ ]:
from graphein.ppi.config import PPIGraphConfig
config = PPIGraphConfig()
config

We also need a list of proteins to work from. Let's use these:

In [ ]:
protein_list = ["CDC42", "CDK1", "KIF23", "PLK1", "RAC2", "RACGAP1", "RHOA", "RHOB"]

Graph construction is handled with the [`compute_ppi_graph()`](https://graphein.ai/modules/graphein.ppi.html#graphein.ppi.graphs.compute_ppi_graph) function that takes a list of proteins, a config for the API calls and `edge_construction_funcs` like [`add_string_edges`](https://graphein.ai/modules/graphein.ppi.html#graphein.ppi.edges.add_string_edges) and [`add_biogrid_edges`](https://graphein.ai/modules/graphein.ppi.html#graphein.ppi.edges.add_biogrid_edges).

In [ ]:
from graphein.ppi.graphs import compute_ppi_graph
from graphein.ppi.edges import add_string_edges, add_biogrid_edges

edge_construction_funcs=[add_string_edges, add_biogrid_edges]

g = compute_ppi_graph(config=config,
                      protein_list=protein_list,
                      edge_construction_funcs=edge_construction_funcs
                     )

## Visualisation
Let's see what it looks like! In this plot node colour & size correspond to the degree. The edge colours indicate the source of interactions:
* Red - STRING
* Blue - BioGrid
* Yellow - Both

In [ ]:
from graphein.ppi.visualisation import plotly_ppi_graph

plotly_ppi_graph(g)

In [ ]:
g.edges(data=True)

## Node Metadata
Cool! Let's add some metadata to our nodes in the form of sequences and UniProt IDs

In [ ]:
from graphein.ppi.features.node_features import add_sequence_to_nodes

g = compute_ppi_graph(config=config,
                      protein_list=protein_list,
                      edge_construction_funcs= edge_construction_funcs,
                      node_annotation_funcs=[add_sequence_to_nodes]
                     )

In [ ]:
for n, d in g.nodes(data=True):
    print(d)

## Graph Metadata
Similarly, you can add graph-level metadata from STRING or BioGrid with:


```python
from graphein.ppi.graph_metadata import add_biogrid_metadata, add_string_metadata, add_string_biogrid_metadata
```

## Structural Interactomics with AlphaFold2

Let's look at a structural interactome! We can add protein graphs to the nodes in our interaction graph. Cool, eh? We think so

In [ ]:
import matplotlib.pyplot as plt
logging.getLogger("matplotlib").setLevel(logging.WARNING)

from graphein.protein.config import ProteinGraphConfig
from graphein.protein.graphs import construct_graph
from graphein.protein.utils import download_alphafold_structure
from graphein.protein.visualisation import plot_protein_structure_graph

pg_config = ProteinGraphConfig()

# Iterate over nodes in PPI Graph
for n, d in g.nodes(data=True):
    try:
        fp = download_alphafold_structure(d['uniprot_ids'][0])[0]
        pg = construct_graph(pg_config, pdb_path=fp)
    
        # Add protein graph as node feature
        d['protein_graph'] = pg
    
        # Plot
        ax = plot_protein_structure_graph(pg, label_node_ids=False, colour_nodes_by="residue_name")
        ax.set_title(d["uniprot_ids"][0])
        plt.show()
    except:
        print(f"Failed to construct graph for {d['uniprot_ids'][0]}")
        continue

Let's checkout the metadata again:

In [ ]:
for n, d in g.nodes(data=True):
    print(d)

## Plot SARS-CoV-2 host-virus proteins (Gordon et al., 2020)

List of human interacting proteins (`protein_list`) retrieved from:

Gordon et al. (2020). A SARS-CoV-2-Human Protein-Protein Interaction Map Reveals Drug Targets and Potential Drug-Repurposing. [[paper](https://www.biorxiv.org/content/10.1101/2020.03.22.002386v2)]

In [ ]:
protein_list = ['AP3B1', 'BRD4', 'BRD2', 'CWC27', 'ZC3H18', 'SLC44A2', 'PMPCB',
       'YIF1A', 'ATP1B1', 'ACADM', 'ETFA', 'STOM', 'GGCX', 'ATP6V1A',
       'PSMD8', 'REEP5', 'PMPCA', 'ANO6', 'PITRM1', 'SLC30A9', 'FASTKD5',
       'SLC30A7', 'TUBGCP3', 'COQ8B', 'SAAL1', 'REEP6', 'INTS4',
       'SLC25A21', 'TUBGCP2', 'TARS2', 'RTN4', 'FAM8A1', 'AASS', 'AKAP8L',
       'AAR2', 'BZW2', 'RRP9', 'PABPC1', 'CSNK2A2', 'CSNK2B', 'G3BP1',
       'PABPC4', 'LARP1', 'FAM98A', 'SNIP1', 'UPF1', 'MOV10', 'G3BP2',
       'DDX21', 'RBM28', 'RPL36', 'GOLGA7', 'ZDHHC5', 'POLA1', 'PRIM1',
       'PRIM2', 'POLA2', 'COLGALT1', 'PKP2', 'AP2A2', 'GFER', 'ERGIC1',
       'AP2M1', 'GRPEL1', 'TBCA', 'SBNO1', 'BCKDK', 'AKAP8', 'MYCBP2',
       'SLU7', 'RIPK1', 'UBAP2L', 'TYSND1', 'PDZD11', 'PRRC2B', 'UBAP2',
       'ZNF318', 'CRTC3', 'USP54', 'ZC3H7A', 'LARP4B', 'RBM41', 'TCF12',
       'PPIL3', 'PLEKHA5', 'TBKBP1', 'CIT', 'HSBP1', 'PCNT', 'CEP43',
       'PRKAR2A', 'PRKACA', 'PRKAR2B', 'RDX', 'CENPF', 'TLE1', 'TLE3',
       'TLE5', 'GOLGA3', 'GOLGA2', 'GOLGB1', 'GRIPAP1', 'CEP350',
       'PDE4DIP', 'CEP135', 'CEP68', 'CNTRL', 'ERC1', 'GCC2', 'CLIP4',
       'NIN', 'CEP112', 'MIPOL1', 'USP13', 'GCC1', 'JAKMIP1', 'CDK5RAP2',
       'AKAP9', 'GORASP1', 'FYCO1', 'C1orf50', 'CEP250', 'TBK1', 'HOOK1',
       'NINL', 'GLA', 'IMPDH2', 'SIRT5', 'NUTF2', 'ARF6', 'RNF41',
       'SLC27A2', 'EIF4E2', 'POR', 'RAP1GDS1', 'WASHC4', 'FKBP15',
       'GIGYF2', 'IDE', 'TIMM10', 'ALG11', 'NUP210', 'TIMM29', 'DNAJC11',
       'TIMM10B', 'TIMM9', 'HDAC2', 'GPX1', 'TRMT1', 'ATP5MG', 'ATP6AP1',
       'SIGMAR1', 'ATP13A3', 'AGPS', 'CYB5B', 'ACSL3', 'CYB5R3', 'RALA',
       'COMT', 'RAB5C', 'RAB7A', 'RAB8A', 'RAB2A', 'RAB10', 'RAB14',
       'RHOA', 'RAB1A', 'GNB1', 'GNG5', 'LMAN2', 'MOGS', 'TOR1AIP1',
       'MTARC1', 'QSOX2', 'HS2ST1', 'NDUFAF2', 'SCCPDH', 'SCARB1',
       'NAT14', 'DCAKD', 'FAM162A', 'DNAJC19', 'SELENOS', 'PTGES2',
       'RAB18', 'MPHOSPH10', 'SRP72', 'ATE1', 'NSD2', 'SRP19', 'SRP54',
       'MRPS25', 'DDX10', 'LARP7', 'MEPCE', 'NGDN', 'EXOSC8', 'NARS2',
       'NOL10', 'CCDC86', 'SEPSECS', 'EXOSC5', 'EXOSC3', 'AATF', 'HECTD1',
       'MRPS2', 'MRPS5', 'EXOSC2', 'MRPS27', 'GTF2F2', 'FBN1', 'FBN2',
       'NUP214', 'NUP62', 'DCAF7', 'EIF4H', 'NUP54', 'MIB1', 'SPART',
       'NEK9', 'ZNF503', 'NUP88', 'NUP58', 'MAT2B', 'FBLN5', 'PPT1',
       'CUL2', 'MAP7D1', 'THTPA', 'ZYG11B', 'TIMM8B', 'RBX1', 'ELOC',
       'ELOB', 'HMOX1', 'TRIM59', 'ARL6IP6', 'VPS39', 'CLCC1', 'VPS11',
       'SUN2', 'ALG5', 'STOML2', 'NUP98', 'RAE1', 'MTCH1', 'HEATR3',
       'MDN1', 'PLOD2', 'TOR1A', 'STC2', 'PLAT', 'ITGB1', 'CISD3',
       'COL6A1', 'PVR', 'DNMT1', 'LOX', 'PCSK6', 'INHBE', 'NPC2', 'MFGE8',
       'OS9', 'NPTX1', 'POGLUT2', 'POGLUT3', 'ERO1B', 'PLD3', 'FOXRED2',
       'CHPF', 'PUSL1', 'EMC1', 'GGH', 'ERLEC1', 'IL17RA', 'NGLY1',
       'HS6ST2', 'SDF2', 'NEU1', 'GDF15', 'TM2D3', 'ERP44', 'EDEM3',
       'SIL1', 'POFUT1', 'SMOC1', 'PLEKHF2', 'FBXL12', 'UGGT2', 'CHPF2',
       'ADAMTS1', 'HYOU1', 'FKBP7', 'ADAM9', 'FKBP10', 'SLC9A3R1',
       'CHMP2A', 'CSDE1', 'TOMM70', 'MARK3', 'MARK2', 'DPH5', 'DCTPP1',
       'MARK1', 'PTBP2', 'BAG5', 'UBXN8', 'GPAA1', 'WFS1', 'ABCC1',
       'F2RL1', 'SCAP', 'DPY19L1', 'TMEM97', 'SLC30A6', 'TAPT1', 'ERMP1',
       'NLRX1', 'RETREG3', 'PIGO', 'FAR2', 'ECSIT', 'ALG8', 'TMEM39B',
       'GHITM', 'ACAD9', 'NDFIP2', 'BCS1L', 'NDUFAF1', 'TMED5', 'NDUFB9',
       'PIGS']

In [ ]:
import networkx as nx
config = PPIGraphConfig()

g = compute_ppi_graph(config=config, protein_list=protein_list, edge_construction_funcs=edge_construction_funcs)
plotly_ppi_graph(g, node_size_multiplier=1, width=1000, height=1000, layout=nx.layout.circular_layout, edge_opacity=0.2)
